# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846900677869                   -0.70    4.8         
  2   -7.852327293746       -2.27       -1.53    1.0   23.2ms
  3   -7.852617125128       -3.54       -2.55    1.5   26.1ms
  4   -7.852645995454       -4.54       -2.89    2.2   30.5ms
  5   -7.852646518440       -6.28       -3.21    1.0   24.2ms
  6   -7.852646680093       -6.79       -4.12    1.2   24.3ms
  7   -7.852646686657       -8.18       -5.10    2.0   29.7ms
  8   -7.852646686726      -10.16       -5.57    1.8   27.2ms
  9   -7.852646686729      -11.55       -5.81    1.2   26.1ms
 10   -7.852646686730      -12.02       -6.76    1.0   24.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846912263954                   -0.70           4.5         
  2   -7.852526732346       -2.25       -1.64   0.80    2.0    245ms
  3   -7.852636813100       -3.96       -2.71   0.80    1.0   21.1ms
  4   -7.852646460395       -5.02       -3.28   0.80    2.2   28.3ms
  5   -7.852646682508       -6.65       -4.13   0.80    1.5   24.5ms
  6   -7.852646686389       -8.41       -4.82   0.80    1.2   22.6ms
  7   -7.852646686723       -9.48       -5.78   0.80    1.8   25.9ms
  8   -7.852646686730      -11.15       -6.70   0.80    2.5   28.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.362998e+01     3.976707e+00
 * time: 0.4465970993041992
     1     1.440284e+00     1.997293e+00
 * time: 0.6391019821166992
     2    -1.595739e+00     2.251622e+00
 * time: 0.6618289947509766
     3    -3.940603e+00     2.056064e+00
 * time: 0.6945400238037109
     4    -5.478939e+00     1.698710e+00
 * time: 0.7270441055297852
     5    -7.018123e+00     9.137941e-01
 * time: 0.7597460746765137
     6    -7.251789e+00     1.158366e+00
 * time: 0.7826390266418457
     7    -7.658926e+00     5.097061e-01
 * time: 0.8057291507720947
     8    -7.743893e+00     3.439463e-01
 * time: 0.8283610343933105
     9    -7.787456e+00     9.210439e-02
 * time: 0.8512201309204102
    10    -7.818644e+00     1.262324e-01
 * time: 0.8738369941711426
    11    -7.833656e+00     1.271061e-01
 * time: 0.8964240550994873
    12    -7.844939e+00     3.434592e-02
 * time: 0.9193220138549805
    13    -7.850543e+00     5.213181e-02
 * time: 0.9421432018

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846788177796                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645886686                   -1.64         
  2   -7.852646686730       -6.10       -3.70    1.61s
  3   -7.852646686730      -13.27       -7.23    159ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.2406990610683587e-7
|ρ_newton - ρ_scfv| = 1.1444935050933841e-7
|ρ_newton - ρ_dm|   = 5.792553560015922e-10
